In [0]:
# NVIDIA GPU 정보 확인
!nvidia-smi

In [0]:
# 먼저 설치 (클러스터 재시작 시 휘발될 수 있으므로 클러스터 라이브러리로 설치 권장)
# %pip install pynvml

# NVML_TEMP_GPU 상수를 pynvml에서 직접 임포트합니다.
from pynvml import * # 또는 from pynvml import nvmlInit, nvmlDeviceGetCount, nvmlDeviceGetHandleByIndex, nvmlDeviceGetName, nvmlDeviceGetUtilizationRates, nvmlDeviceGetMemoryInfo, nvmlDeviceGetTemperature, NVML_TEMP_GPU, NVMLError, nvmlShutdown

try:
    nvmlInit()
    deviceCount = nvmlDeviceGetCount()
    print(f"발견된 GPU 개수: {deviceCount}")

    for i in range(deviceCount):
        handle = nvmlDeviceGetHandleByIndex(i)
        deviceName = nvmlDeviceGetName(handle)
        utilization = nvmlDeviceGetUtilizationRates(handle)
        memory = nvmlDeviceGetMemoryInfo(handle)
        
        # NVML_TEMP_GPU 상수를 사용하여 온도 정보를 가져옵니다.
        #temperature = nvmlDeviceGetTemperature(handle, NVML_TEMP_GPU)

        #print(f"\n--- GPU {i}: {deviceName.decode('utf-8')} ---")
        print(f"  활용률: {utilization.gpu}%")
        print(f"  메모리 사용률: {memory.used / memory.total * 100:.2f}% ({memory.used / (1024**3):.2f} / {memory.total / (1024**3):.2f} GB)")
        #print(f"  온도: {temperature} C")

except NVMLError as error:
    print(f"NVML 에러: {error}")
finally:
    try:
        nvmlShutdown()
    except NVMLError as error:
        print(f"NVML 종료 에러: {error}")


In [0]:
# CUDA 버전 확인
!nvcc --version

# 또는 Python에서
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"GPU count: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")

In [0]:
# cuDF 설치 및 작동 확인
try:
    import cudf
    import cuml
    print("✅ RAPIDS cuDF/cuML 사용 가능")
    
    # 간단한 GPU 연산 테스트
    test_df = cudf.DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6]})
    print(f"cuDF 테스트: {test_df.sum()}")
    
except ImportError as e:
    print(f"❌ RAPIDS 라이브러리 없음: {e}")
    print("설치 필요: %pip install cudf-cu11 cuml-cu11")

In [0]:
import cudf
gdf = cudf.DataFrame({"a":[1,2,3],"b":[4,5,6]})
print(gdf)


In [0]:
# 기본 설치된 GPU 라이브러리 확인
import pkg_resources
gpu_packages = [pkg for pkg in pkg_resources.working_set 
                if any(keyword in pkg.project_name.lower() 
                      for keyword in ['cuda', 'cudf', 'cuml', 'rapids'])]

print("설치된 GPU 관련 패키지:")
for pkg in gpu_packages:
    print(f"  {pkg.project_name}: {pkg.version}")

In [0]:
# 현재 Spark 설정 확인
spark_configs = spark.sparkContext.getConf().getAll()
gpu_configs = [config for config in spark_configs if 'gpu' in config[0].lower() or 'rapids' in config[0].lower()]

print("GPU 관련 Spark 설정:")
for key, value in gpu_configs:
    print(f"{key}: {value}")

In [0]:
# 클러스터 정보 확인
print("Spark 버전:", spark.version)
print("Python 버전:", sys.version)

# 드라이버와 워커 정보
#print("드라이버 메모리:", spark.conf.get("spark.driver.memory"))
print("실행자 메모리:", spark.conf.get("spark.executor.memory"))

In [0]:
!pip list

In [0]:
# PyTorch 설치 여부 확인
try:
    import torch
    print(f"✅ PyTorch 이미 설치됨: {torch.__version__}")
    print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"CUDA 버전: {torch.version.cuda}")
        print(f"GPU 개수: {torch.cuda.device_count()}")
except ImportError:
    print("❌ PyTorch 설치되지 않음")